In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def calculate_ma(series, window=30):
    """Calculate moving average for a series"""
    return pd.to_numeric(series, errors='coerce').rolling(window=window).mean()

def format_value(value, raw_unit=True):
    """
    Format values based on their size
    Parameters:
    - value: numeric value to format
    - raw_unit: if True, return just the number and unit; if False, include '$' prefix
    """
    try:
        value = float(value)
        abs_value = abs(value)
        
        if abs_value >= 1e12:
            formatted = f"{value/1e12:.2f}T"
        elif abs_value >= 1e9:
            formatted = f"{value/1e9:.2f}B"
        elif abs_value >= 1e6:
            formatted = f"{value/1e6:.1f}M"
        elif abs_value >= 1e3:
            formatted = f"{value/1e3:.0f}K"
        else:
            formatted = f"{value:.0f}"
            
        return formatted if raw_unit else f"${formatted}"
    except (ValueError, TypeError):
        return "N/A"

def get_scale_and_prefix(max_value):
    """
    Determine the appropriate scale and prefix for the y-axis based on the maximum value
    """
    abs_value = abs(max_value)
    if abs_value >= 1e12:
        return 1e12, "T"
    elif abs_value >= 1e9:
        return 1e9, "B"
    elif abs_value >= 1e6:
        return 1e6, "M"
    elif abs_value >= 1e3:
        return 1e3, "K"
    return 1, ""

def get_and_plot_market_caps(ma_window=30):
    """
    Fetch and plot cryptocurrency market caps with configurable options
    
    Parameters:
    - ma_window: int, window size for moving average calculation (default 30 days)
    """
    # Read the CSV file
    df = pd.read_csv('coinmarketcap_20190106_top_125.csv')
    
    # Create symbol mapping dictionary
    symbol_mapping = {
        row['Name']: f"{row['Symbol']}-USD"
        for _, row in df.iterrows()
    }
    
    # Dictionary to store market caps for each crypto
    market_caps_dict = {}
    
    # Define the date range
    start_date = '2019-01-01'
    end_date = datetime.now().strftime('%Y-%m-%d')
    
    # Loop through each cryptocurrency
    for name, symbol in symbol_mapping.items():
        try:
            print(f"Fetching data for {name} ({symbol})...")
            
            # Fetch data from Yahoo Finance
            ticker = yf.Ticker(symbol)
            hist = ticker.history(start=start_date, end=end_date)
            
            if not hist.empty:
                # Calculate market cap (Close Price * Volume)
                market_caps_dict[name] = pd.to_numeric(hist['Close'] * hist['Volume'], errors='coerce')
            
            # Add a small delay to avoid hitting rate limits
            time.sleep(1)
            
        except Exception as e:
            print(f"Error fetching data for {name} ({symbol}): {str(e)}")
            continue
    
    # Combine all market caps into a single DataFrame
    if market_caps_dict:
        all_market_caps = pd.concat(market_caps_dict, axis=1)
        
        # Convert all values to numeric and handle missing values
        df_filled = all_market_caps.apply(pd.to_numeric, errors='coerce').ffill()
        
        # Calculate market metrics
        others_market_cap = df_filled.drop('Bitcoin', axis=1).sum(axis=1)
        btc_dominance = (df_filled['Bitcoin'] / (df_filled['Bitcoin'] + others_market_cap)) * 100
        
        # Calculate moving averages
        others_ma = calculate_ma(others_market_cap, ma_window)
        dominance_ma = calculate_ma(btc_dominance, ma_window)
        
        # Get appropriate scale for the market cap values
        scale, unit_prefix = get_scale_and_prefix(others_market_cap.max())
        
        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        # Add raw data traces
        fig.add_trace(
            go.Scatter(
                x=others_market_cap.index,
                y=others_market_cap.astype(float) / scale,
                name="Total Market Cap (excl. BTC)",
                mode='lines',
                line=dict(color='#000000', width=1),
                hovertemplate=f"Total Market Cap: $%{{y:.2f}}{unit_prefix}<extra></extra>"
            ),
            secondary_y=False
        )

        # Add MA for market cap
        fig.add_trace(
            go.Scatter(
                x=others_ma.index,
                y=others_ma.astype(float) / scale,
                name=f"{ma_window}D MA - Market Cap",
                mode='lines',
                line=dict(color='#000000', width=2, dash='dash'),
                hovertemplate=f"{ma_window}D MA: $%{{y:.2f}}{unit_prefix}<extra></extra>"
            ),
            secondary_y=False
        )

        # Add BTC dominance trace
        fig.add_trace(
            go.Scatter(
                x=btc_dominance.index,
                y=btc_dominance.astype(float),
                name="BTC Dominance",
                mode='lines',
                line=dict(color='#666666', width=1),
                hovertemplate="BTC Dominance: %{y:.1f}%<extra></extra>"
            ),
            secondary_y=True
        )

        # Add MA for dominance
        fig.add_trace(
            go.Scatter(
                x=dominance_ma.index,
                y=dominance_ma.astype(float),
                name=f"{ma_window}D MA - Dominance",
                mode='lines',
                line=dict(color='#666666', width=2, dash='dash'),
                hovertemplate=f"{ma_window}D MA: %{{y:.1f}}%<extra></extra>"
            ),
            secondary_y=True
        )

        # Update layout
        fig.update_layout(
            title=dict(
                text=f'Cryptocurrency Market Cap vs Bitcoin Dominance',
                font=dict(
                    color='#000000',
                    size=29,
                    weight='bold'
                ),
                x=0.055,
                y=0.94,
                xanchor='left',
                yanchor='top'
            ),
            width=1350,
            height=750,
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="top",
                y=0.98,
                xanchor="left",
                x=0.02,
                font=dict(
                    color='#000000',
                    size=17
                ),
                bgcolor='rgba(255, 255, 255, 1)'
            ),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        # Convert index to datetime if it's not already
        if not isinstance(others_market_cap.index, pd.DatetimeIndex):
            others_market_cap.index = pd.to_datetime(others_market_cap.index)

        # Update x-axis to show every 3 months
        date_range = pd.date_range(
            start=others_market_cap.index.min(), 
            end=others_market_cap.index.max(), 
            freq='3ME'
        )
        
        # Create two-line date labels
        date_labels = [f"{d.strftime('%b')}<br>{d.strftime('%Y')}" for d in date_range]
        
        fig.update_xaxes(
            ticktext=date_labels,
            tickvals=date_range,
            tickangle=0,
            showgrid=False,
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            tickfont=dict(
                color='#000000',
                size=17
            ),
            linecolor='#000000',
            linewidth=1,
            mirror=True
        )

        # Calculate y-axis ranges and tick values
        y_max_market = np.ceil(others_market_cap.max() / scale / 10) * 10
        y_tick_vals_market = np.linspace(0, y_max_market, 10)
        
        y_max_dom = np.ceil(btc_dominance.max() / 10) * 10
        y_tick_vals_dom = np.linspace(0, y_max_dom, 10)

        # Update primary y-axis (Market Cap)
        fig.update_yaxes(
            title=None,
            showgrid=False,
            tickfont=dict(
                color='#000000',
                size=17
            ),
            ticktext=[f"${val:.0f}{unit_prefix}" for val in y_tick_vals_market],
            tickvals=y_tick_vals_market,
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[0, y_max_market],
            secondary_y=False,
            linecolor='#000000',
            linewidth=1,
            mirror=True
        )

        # Update secondary y-axis (Dominance)
        fig.update_yaxes(
            title=None,
            showgrid=False,
            tickfont=dict(
                color='#000000',
                size=17
            ),
            ticksuffix="%",
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[0, y_max_dom],
            secondary_y=True,
            linecolor='#000000',
            linewidth=1,
            mirror=True
        )

        # Save outputs
        fig.write_html("crypto_dominance.html")
        pd.DataFrame({
            'Total Market Cap (excl. BTC)': others_market_cap,
            'BTC Dominance (%)': btc_dominance,
            f'Market Cap {ma_window}D MA': others_ma,
            f'Dominance {ma_window}D MA': dominance_ma
        }).to_csv('dominance_analysis.csv')
        
        print("Analysis completed! Check 'crypto_dominance.html' and 'dominance_analysis.csv' for results.")
        
        # Show the plot
        fig.show()
    else:
        print("No data was collected")

if __name__ == "__main__":
    get_and_plot_market_caps(ma_window=30)  # 30-day MA

Fetching data for Bitcoin (BTC-USD)...
Fetching data for Ethereum (ETH-USD)...
Fetching data for XRP (XRP-USD)...
Fetching data for Bitcoin Cash (BCH-USD)...
Fetching data for EOS (EOS-USD)...
Fetching data for Litecoin (LTC-USD)...
Fetching data for Stellar (XLM-USD)...
Fetching data for Tether (USDT-USD)...
Fetching data for Bitcoin SV (BSV-USD)...
Fetching data for TRON (TRX-USD)...
Fetching data for Cardano (ADA-USD)...
Fetching data for IOTA (MIOTA-USD)...
Fetching data for Monero (XMR-USD)...
Fetching data for Binance Coin (BNB-USD)...
Fetching data for Dash (DASH-USD)...
Fetching data for NEM (XEM-USD)...
Fetching data for Ethereum Classic (ETC-USD)...
Fetching data for Neo (NEO-USD)...
Fetching data for Maker (MKR-USD)...
Fetching data for Zcash (ZEC-USD)...
Fetching data for Waves (WAVES-USD)...
Fetching data for USD Coin (USDC-USD)...
Fetching data for Tezos (XTZ-USD)...
Fetching data for Dogecoin (DOGE-USD)...
Fetching data for Bitcoin Gold (BTG-USD)...
Fetching data for VeC

$PAX-USD: possibly delisted; no timezone found


Fetching data for BitShares (BTS-USD)...
Fetching data for Verge (XVG-USD)...
Fetching data for Siacoin (SC-USD)...
Fetching data for Stratis (STRAT-USD)...
Fetching data for Augur (REP-USD)...
Fetching data for Aeternity (AE-USD)...
Fetching data for Steem (STEEM-USD)...
Fetching data for Gemini Dollar (GUSD-USD)...
Fetching data for Komodo (KMD-USD)...
Fetching data for Populous (PPT-USD)...
Fetching data for Bytom (BTM-USD)...
Fetching data for Buggyra Coin Zero (BCZERO-USD)...


$BCZERO-USD: possibly delisted; no timezone found


Fetching data for Pundi X (NPXS-USD)...
Fetching data for IOST (IOST-USD)...
Fetching data for Holo (HOT-USD)...
Fetching data for Aurora (AOA-USD)...
Fetching data for Single Collateral DAI (SAI-USD)...
Fetching data for Golem (GNT-USD)...
Fetching data for Factom (FCT-USD)...
Fetching data for Status (SNT-USD)...
Fetching data for Electroneum (ETN-USD)...
Fetching data for Dentacoin (DCN-USD)...
Fetching data for Cryptonex (CNX-USD)...
Fetching data for MaidSafeCoin (MAID-USD)...
Fetching data for ODEM (ODE-USD)...
Fetching data for Ardor (ARDR-USD)...
Fetching data for Huobi Token (HT-USD)...
Fetching data for KuCoin Shares (KCS-USD)...
Fetching data for REPO (REPO-USD)...
Fetching data for PIVX (PIVX-USD)...
Fetching data for Ark (ARK-USD)...
Fetching data for Waltonchain (WTC-USD)...
Fetching data for Decentraland (MANA-USD)...
Fetching data for Insight Chain (INB-USD)...
Fetching data for Aion (AION-USD)...
Fetching data for Nexo (NEXO-USD)...
Fetching data for DigixDAO (DGD-USD)